#### This file creates correctness.csv, which stores the correctness marks of each student

In [ ]:
import os
import math
import numpy as np
import pandas as pd 

In [ ]:
home_dir: str = os.getcwd()
s_id, e_id = 0, 16
print (home_dir)

In [ ]:
weights = {'small': 50, 'medium': 35, 'large': 15}
categories = {'small': [0, 1, 2, 3, 4, 5], 'medium': [6, 10, 12, 14], 'large': [15]}
marksFor = dict()

In [ ]:
s = []
s = os.listdir(os.path.join(home_dir, 'sub'))
all_students = [student.split('_')[1] for student in s]
paired_to = {s: '' for s in all_students}

In [ ]:
df = pd.read_csv(os.path.join(home_dir, 'studentPairs.csv'))
for _, row in df.iterrows():
    paired_to[row['student1']] = row['student2']

In [ ]:
range_of_k = dict()
for testid in range(s_id, e_id):
    test_dir = os.path.join(home_dir, f'test{testid}')
    if os.path.exists(test_dir):
        info_file = (os.path.join(test_dir, 'task1_info.txt'))
        startk, endk = None, None
        with open(info_file, 'r') as f:
            line: str = f.readline().split(',')
            startk: int = int((line[0]).split('=')[1])
            endk: int = int((line[1]).split('=')[1])
            f.close()
        range_of_k[testid] = (startk, endk)

print (range_of_k)

In [ ]:
for category, ids in categories.items():
    uniform_weight = weights[category] / len(ids)
    for testid in ids:
        marksFor[testid] = uniform_weight

test_cases = list(marksFor.keys())
student_df = pd.DataFrame(columns= ['Entry_no'] + test_cases + ['Total Score'])
student_df['Entry_no'] = all_students
student_df.set_index('Entry_no', inplace=True)
student_df = student_df.replace(np.nan, 0)
student_df

In [ ]:
# for each k within the test case.
final_marks = dict()
for testid, (startk, endk) in range_of_k.items():
    if not(testid in marksFor):
        continue
    marks = marksFor[testid]
    total_subpart = endk - startk + 1
    subpart_marks = marks / total_subpart
    final_marks[testid] = dict()
    for k in range(startk, endk + 1):
        final_marks[testid][k] = subpart_marks


In [ ]:
student_marks = {k:dict() for k in test_cases}

for testid in test_cases:
    logs_dir = os.path.join(home_dir, f'logs_{testid}')
    if os.path.exists(logs_dir):
        verdict_file = os.path.join(logs_dir, f'time_test_{testid}.csv')
        df = pd.read_csv(verdict_file)
        startk, endk = range_of_k[testid]
        columns = [f'Verdict GroupSize={k}' for k in range(startk, endk + 1)]
        for idx, row in df.iterrows():
            entry_no = row['Entry_number']
            _entry_no = entry_no.replace('.txt', '')
            sum = 0
            for k in columns:
                _k = int(k.split('=')[1])
                if row[k] == 'AC':
                    sum += float(final_marks[testid][_k])
                if row[k] == 'PC':
                    sum += float(final_marks[testid][_k] / 2)
            student_marks[testid][_entry_no] = sum
        
for testid, scores in student_marks.items():
    for eno, s in scores.items():
        student_df.loc[eno, testid] = s

cols = list(student_df.columns)
cols.remove('Total Score')
student_df['Total Score'] = student_df.loc[:, cols].sum(axis='columns')
student_df.to_csv(os.path.join(home_dir,'correctness.csv'))
student_df